#Import Library


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

#Data Processing

In [2]:
# Loading and Preparing Dataset
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
class MNIST_Dataset(Dataset):
    
    def __init__(self, X, y, transform = transforms.Compose(
                     [transforms.ToPILImage(), 
                      transforms.ToTensor(), 
                      transforms.Normalize(mean=(0.5,), std=(0.5,))])):
            
        self.transform = transform
        self.X = X.reshape((-1,56,56)).astype(np.uint8)[:,:,:,None]
        self.y = y
        
    @classmethod
    def from_train(cls, file_path_images, file_path_labels):
        train_images = pd.read_pickle(file_path_images)
        train_labels = pd.read_csv(file_path_labels)
        
        return cls(X=train_images, y=torch.from_numpy(train_labels['Category'].values))
        
    @classmethod
    def from_test(cls, file_path_images):
        test_images = pd.read_pickle(file_path_images)
        
        return cls(X=test_images, y=None)
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.transform(self.X[idx])
        # X = X.to(device)
        if self.y is not None:
            return X, self.y[idx]   #.to(device)
        else:
            return X

In [4]:
train_data = pd.read_pickle('images_l.pkl')
train_labels = pd.read_pickle('labels_l.pkl')
test_data = pd.read_pickle('images_test.pkl')

In [5]:
number_train_images = int(train_data.shape[0] * 0.9)

X_train = train_data[:number_train_images]
y_train = train_labels[:number_train_images]
print(X_train.shape)
print(y_train.shape)

X_validation = train_data[number_train_images:]
y_validation = train_labels[number_train_images:]
print(X_validation.shape)
print(y_validation.shape)
print(test_data.shape)

(27000, 56, 56)
(27000, 36)
(3000, 56, 56)
(3000, 36)
(15000, 56, 56)


In [7]:
batch_size = 32

train_dataset = MNIST_Dataset(X_train, y_train)
validation_dataset = MNIST_Dataset(X_validation, None)
test_dataset = MNIST_Dataset.from_test('images_test.pkl')

In [8]:
# Loading
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,
                                           batch_size=batch_size, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size, 
                                           shuffle=False)

#ResNet

In [9]:
class block(nn.Module):
    def __init__(
        self, in_channels, out_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size=1, stride=1, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            out_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(
            out_channels,
            out_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

In [10]:
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], out_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], out_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], out_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], out_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != out_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    out_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(out_channels * 4),
            )

        layers.append(
            block(self.in_channels, out_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = out_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as out layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

In [11]:
def ResNet50(img_channel=3, num_classes=36):
    return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)


def ResNet101(img_channel=3, num_classes=36):
    return ResNet(block, [3, 4, 23, 3], img_channel, num_classes)


def ResNet152(img_channel=3, num_classes=36):
    return ResNet(block, [3, 8, 36, 3], img_channel, num_classes)


#Prediction

In [12]:
def predict(model, data_loader):
    real=[]
    for i, data in enumerate(data_loader):
        output = model(data)
        for i in range(output.shape[0]):
            com = onehot(output[i])
            real.append(com)   
    return torch.Tensor(real)

In [13]:
def softMax(vector):
    e = np.exp(vector)
    return e / e.sum()

In [14]:
def onehot(vector):
    digit=vector[0:10]
    alphabet=vector[10:36]
    max_1=torch.argmax(digit,0,keepdim=True)
    max_2=torch.argmax(alphabet,0,keepdim=True)
    one_hot1=torch.FloatTensor(digit.shape)
    one_hot2=torch.FloatTensor(alphabet.shape)
    one_hot1.zero_()
    one_hot2.zero_()
    one_hot1.scatter_(0, max_1, 1)
    one_hot2.scatter_(0, max_2, 1)
    return torch.cat((one_hot1,one_hot2),0).tolist()

#Training

In [15]:
def train_model(model_class, validation_loader, y_validation, dropout=0.5, epochs=30, criterion=nn.CrossEntropyLoss(), learning_rate=1e-3):
    input_dim = 56 * 56
    output_dim = 36
    model = model_class(input_dim, output_dim)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    losses_per_epoch = []
    validation_score_per_epoch = []
    for epoch in range(epochs):
    
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
#             labels = labels.to(dtype=torch.long)
            optimizer.zero_grad()

            outputs = model(inputs) #### 1:00
#             real=[]
#             for i in range(outputs.shape[0]):
#                 com = onehot(outputs[i])
#                 real.append(com.tolist())  
            loss = criterion(outputs, labels)#torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()
        
        model.eval()
       
        validation_predictions = predict(model, validation_loader).numpy()
#         y_val = np.argmax(y_validation, axis = 1)
       
        validation_score_per_epoch.append(accuracy_score(y_validation.tolist(), validation_predictions.tolist()))
        model.train()
        
        losses_per_epoch.append(loss.item())
    print('[Finished Training] Score: %.4f Loss: %.4f' % (validation_score_per_epoch[-1], losses_per_epoch[-1]))
    return model, losses_per_epoch, validation_score_per_epoch

#ModelModel Comparision

In [16]:
summary_50_resnet = {'model' : [], 'loss' : [], 'score' : []}
model, loss, score = train_model(ResNet50, validation_loader, y_validation, dropout=0.1, epochs=20)
summary_50_resnet['model'].append(model)
summary_50_resnet['loss'].append(loss)
summary_50_resnet['score'].append(score)

RuntimeError: Given groups=1, weight of size [64, 3136, 7, 7], expected input[32, 1, 56, 56] to have 3136 channels, but got 1 channels instead